# Airbnb Reviews Dataset

The dataset you'll be working with in this exercise is a sample of Airbnb Review data. Airbnb provides all kinds of data concerning quarterly or monthly data of residences in big cities all around the world. The data can be downloaded for free at <a href="http://insideairbnb.com/get-the-data/">inside airbnb</a>.

The part of the data relevant to this exercise, as mentioned above, relates to reviews. Because of airbnb's generousity the review datasets for the different cities contain millions of entries which might be a little too much for your first attempt at analyzing text only.

For this reason a random subset of 25,000 entries has been prepared for you. In comparison to the original dataset this might seem like a small dataset but some of the following steps to analyze this data might take a couple of seconds or even minutes to compute, depending on your hardware.

The dataset in `sample_reviews.csv` is a subset of reviews of airbnbs in the city of Berlin. It also has been prefiltered to only contain reviews which could be identified as English by <a href="https://pypi.org/project/langdetect/"> Nakatani Shuyo's language detection library</a>.

### Exercise:

- Import necessary package to read `sample_reviews.csv`
- Read `sample_reviews.csv` and create a `pandas.DataFrame` with the review data. Note that it has an integer index in its first column.
- Inspect the data using the DataFrame's functionality.

In [ ]:
#Import necessary package
import pandas as pd

#Read sample of review data
df = ___________

#Inspect the data by using the DataFrame's functionality
print(__________)
print(__________)
print(__________)


# Vectorizing Text Data

As you might have guessed already the only column of the `sample_reviews` data is the `comments` column which contains the text of a given review. As the machine learning models you got to know over the time of the semester don't handle blobs of text as only input very well some kind of further transformation is needed.

One of the less complex ways to represent blobs of text in a way a machine can understand is to utilize CountVectorization. This technique seperates the different words of a text from each other and counts their occurence in the given text. At this point it is important to know in which language the text you will be transforming is written. If you don't know the language it can be detected  by language detectors like the one mentioned above. In this case the language recognition and filtering has already been done for you. All reviews in the data were detected as English.

The next step after figuring out the language is to delete stop words from the texts. This is a very important step in almost all analysis of natural language. Stop words are the words that are most frequently used in a language and therefore might not be really carrying a lot of information to analyse. Some examples of stop words in the English language are: `the`, `is`, `which` or `on`. These words are helpful for humans to communicate information fluently and more understandable but in most cases do not carry enough information to be of high importance for a machine learning algorithmn.

The goal of this analysis is clustering the airbnb reviews, so what's needed is the core information of a review which does not include stop words. Another argument to delte stop words is that because of their frequency they might not be a good indicator to seperate the data anyway.

One last important concept is the `max_features` attribute of the `CountVectorizer`. It limits the number of dimensions of the transformed dataset to a specified amount and only lists the most frequent words in a text aside from the stopwords. This is important when transforming text data because there is really no theoretical upper bound to how many different words occur in a text. This can lead to enormous numbers of features in the transformed dataset which then increases the risk of facing the negative impact of the curse of dimensionality.

### Exercise
- Import `CountVectorizer` from scikit-learn's `feature_extraction.text` module.
- Initialize the `CountVectorizer` with `english` stop words and a maximum number of `100` features.
- Fit the `CountVectorizer` to the data and transform in the same line using `fit_transform()`. Pass only the `comments` column of the DataFrame containing the reviews.
- Save the feature names in variable `words` by calling the `CountVectorizer`'s `get_feature_names_out()` function.
- Create a `pd.DataFrame` for the transformed features using the output of the `CountVectorizer` and calling its `.toarray()` function as data. Pass the feature names variable to the `columns` argument.
- Print the first few rows of the new DataFrame to see if it worked.

In [ ]:
#Import CountVectorizer
____________________

#Initialize CountVectorizer
count_vect = _______________(stop_words=________,max_features=______)

#Apply CountVectorizer
data_count = _____________.fit_transform(__________)

#Create DataFrame
words = ______________
data = pd.DataFrame(__________.toarray(),_____________)

#Print first few rows of vectorized review data
____________


# Dimensionality Reduction

Although we already set the `CountVectorizer`'s `max_feature` argument to just the hundred most frequent words in the corpus of reviews, hundred dimensions are a lot of dimensions and will probably increase the computation time of any given clustering algorithmn because distance needs to be calculated in every dimension.

A way of dealing with this is reducing the dimensionality of a dataset but preserving as much information as possible. Probably one of the oldest methods (invented in 1901 by Karl Pearson) that is still used in data science is called `Principal Components Analysis (PCA)`. A key part of its longevity is its strong theoretical foundation and its efficency in terms of processing time. It transforms metrically scaled data into a set of principal components. These resulting principal components are sorted by the amount of variance they explain in the original data. Then only the n components of these are selected that explain the most or a desired ratio (e.g. 95 percent or 90 percent) of the variance of the original dataset.

As you might have noticed the PCA interprets the information in a dataset as its variance and therefore aims to represent as much of the variance of the original dataset as possible in the reduced dataset. One of the downsides of this technique is that it only operates on the level of correlations in the dataset which means only linear relationships in the original data can be modelled.

The computational efficiency is due to the calculation of principal components being a singular value decomposition which is a fairly simple matrix multiplication of the original data's variance-covariance matrix and the original data matrix itself. Those types of computations can be implemented really efficiently using `numpy` and it's under the hood `C` calculations.

An important step before utilizing the `Principal Components Analysis` is scaling the data by using e.g. `sklearn`'s standard scaler.

### Exercise
- Import `PCA` from `sklearn`'s `decomposition` module.
- Initialize a `PCA` object. You want it to output the `10` components that explain the most variance in the original data. Also set the `random_state` to `123` for reproducibility.
- Fit the `PCA` object to the count vectorized data.
- Transform the count vectorized data into the ten principal components that explain the most of its variance and save the result to a new `pd.DataFrame`.
- Print the array of explained variance ratio and its sum using the `PCA` objects `explained_variance_ratio_` attribute.
- Print the first few rows of the reduced dataset to see if it worked.
- Can explain why you could use the `PCA` in this dataset although it represents text, which is not metrically scaled?
- Why didn't you need to use the `StandardScaler` before utilizing the `PCA`'s `.fit()` and `.transform()` methods on the dataset?

In [ ]:
#Import PCA
_____________________

#Initialize PCA object for 10 Principal Components
pca = _____(n_components=_____,____________)

#Fit PCA object to vectorized review data
pca.fit(data)
data_reduced = __________(______.transform(data))

#Print amount of explainend variance by first 10 principal components
print(_______________________)
print(_______________________.sum())

#Print first few rows of reduced data
_______________________

# Empty Markdown for your answers

# K-Means Clustering

As already mentioned the target of this analysis is clustering the sample of Airbnb reviews in the city of Berlin and interpreting the results. Clustering is, as well as most algorithmns for dimensionality reduction, part of the machine learning discipline of `unsupervised learning`. That means that the dataset does not contain values of the target variable.

You can imagine the analysis you are doing right now as a classification problem but you don't have any information about the affiliation to a class of any of the dataset entries. The best way to deal with this is to take an educated guess on the number of classes and the affiliation of every dataset to theses classes.

One way to take that guess is by determining a number of classes `k` and choosing `k` random points in the dataset. The next step is to calculate the distances between every entry of the dataset and the random points and assigning each dataset entry to the random point it has the lowest sum of distances (one distance for every feature) to. The third step is updating the random `k` points by moving it to the point in the dataset where it has to lowest sum of distances to all the point that were assigned to it in the previous step. It is now the `clustering center` or `centroid` of these points. The second and thiord step are then repeated until the `centroids` aren't updated anymore or a number of maximal iterations is reached. 

A method to improve the initial guess of random points is to plot a clustering metrics' values to different possible values for `k` as the so called `elbow plot`. The x-axes shows the possible values for `k` and the y-axes shows the value of the clustering metric e.g. `Silhouette Score`. The best number for `k` is then determined by looking for the most obvious bend in the resulting line.

The above mentioned metric of `Silhouette Score` or `Silhouette Coefficient` is a ratio of distance between the mean intra cluster distance and the distance between a datapoint and the nearest cluster it is not a part of. Most of the other clustering metrics use some kind of distance ratio in-between datapoints in a cluster and between clusters or their centroids to determine if one cluster result is better than the other.

### Exercise:

- Import `KMeans` from scikit-learn's `cluster` module.
- Import `silhouette_score` and `calinski_harabasz_score` from `sklearn.metrics`.
- Import `matplotlib.pyplot`.
- Complete the loop to calculate the silhouette scores for k = [2,3,4,5,6] and save it to a list.
- Call `matplotlib.pyplot`'s `.subplots()` function and set the figsize width to `16` inches and the height to `10` inches. Give the `title`,`xlabel` and `ylabel` fitting values.
- Create the same elbow plot using the `calinski_harabasz_score`
- Choose a `k` for fitting the k-means algorithmn on the data based on the implications of the elbow plots.
- Fit the `KMeans` object with the chosen `k` on the reduced dataset.
- Explain what ratio `calinksi_haberasz_score` calculates and how it differs from silhouette score.
- Can you explain what the `max_iter` argument when initializing a k-means object does?

In [ ]:
#Imports
___________________
___________________
___________________

#Initialize k-means clustering object
kmeans = KMeans()

#Elbow Plot for k in kmeans with silhouette score
silhouette_scores = []

for k in range(_____):
    km = KMeans(n_clusters=k, 
                max_iter=300, 
                tol=1e-04, 
                init='k-means++', 
                n_init=10, 
                random_state=42, 
                algorithm='lloyd')
    km.fit(data_reduced)
    _______________.append(silhouette_score(data, km.labels_))

#Show Plot and Set labels
fig, ax = plt.subplots(____________)
ax.plot(range(2, 7), silhouette_scores, 'bx-')
ax.set_title(_______________)
ax.set_xlabel(_____________)
ax.set_ylabel(_______________)
plt.xticks(range(2, 7))
plt.tight_layout()
_____________

#Elbow Plot for k in kmeans with calinski haberasz score
ch_scores = []








#Show Plot and Set labels









#Choose k for K-Means algorithmn
km = KMeans(n_clusters=____, 
                max_iter=300, 
                tol=1e-04, 
                init='k-means++', 
                n_init=10, 
                random_state=42, 
                algorithm='lloyd')

#Fit k-means model
km_fitted = km.fit(__________)

# Empty Markdown for your answers

# Visualizing Clustering Results

Other than calculating clustering metrics there is only one other way to calculate the quality of the results. That is visualizing clustering results and checking if the result seem to be following a natural pattern in the data.

Of course one of the main problems of this is data with more than three dimensions is that it can not be visualized in a conventional python plot. The answer is of course dimensionality reduction. For the analysis you are doing right now it is most practical to use `PCA` because that is the only dimensionality reduction technique you know at this point.

Note that plain old `PCA`, although being efficient and well justified theoratically, is not considered the best method to visualize high dimensional data. When interpreting visualizations of principal components you need to remember it only considers linear dependencies. `PCA` transforms the data into perpendicular principal components which can, especially when only visualizing two components in a plot, cause major distortions of the real relationships in the dataset. Some of the more sophisticated visualization methods that can visualize even nonlinear relationships of highdimensional data are `Kernel PCA` or `t-sne`.

For now let's settle on visualizign the first two principal components colored by the cluster they were assigned to by `KMeans`.

### Exercise:
- Import `seaborn` and `numpy`.
- Initialize a `PCA` object that reduces data to only two principal components.
- Fit and transform the review data utilizing the ´PCA´ objects' methods.
- Create a `pd.DataFrame` from the reduced dataset and give the columns fitting names.
- Concatenate the clustering result to the DataFrame utilizing the `KMeans` objects' `.labels_` attribute. Give the new column a fitting name.
- Give the scatterplot a fitting title and set its width to `16` and its height to `10`.
- Create and display a `seaborn.scatterplot` by passing the right column names to the `x`,`y` and `hue` arguments. Also pass the DataFrame you just created to the `data` argument.
- Complete the loop to print for random reviews in every cluster. Pass the length of the dataset to `numpy`'s `random.choice` function.
- Complete the code to create a barplot of the amount of words in every cluster.
- Interpret the plots and evaluate results. Did you get a good clustering result? Can you see any promising patterns? Either way give reasons for your answer.
- Can you think of a kind of analysis that might have been more reasonable regarding the fact that the analysed dataset consists only of blobs of text?

In [ ]:
#Imports
import seaborn as sns
import numpy as np

#Reduce dimensionality for plotting the clustering result
pca = _____(n_components=____,___________)
pca.fit(data)
reduced_data = ____________

#Create a visualization DataFrame of the two principal components and the clustering results
visualizing_data = pd.DataFrame(_____________, 
                                columns=[__________________,_________________])

target = pd.DataFrame(km_fitted.labels_,columns=[____________])
visualizing_data = pd.concat([visualizing_data,target],axis=1)

#Create Scatterplot of first two principal components colored by clustering results
plt.figure(_______________)
plt.title(________________)
sns.scatterplot(x=________________,y=______________________,hue=____________,data=_______________)
plt.legend()
plt.tight_layout()
plt.show()

#Get unique values for clustering results
clusters = _______________________

#Print four random examples of each cluster
for i in clusters:

    counter = 0

    while counter <= ______:
        
        #Choose random index to get samples of clustering result
        idx = np.random.choice(len(_______),1)

        #Get value on random location
        v = __________[__________].iloc[idx].values[0]

        while v != i:

            #Get new index until value at target[idx] is i
            idx = ______________
            v = _________________

        #Print random instance
        print('Data of sample: ', df.iloc[idx][________].values[0])
        print('Cluster of sample: ', i)

        counter ________
    
    print('')

#Create Barplot of sum of words of different clusters
data_with_clusters = pd.concat([data,target],axis=1)
colors=['green','blue','orange','yellow','red']

for i in clusters:

    #Subset data of only one cluster
    temp_df = data_with_clusters.loc[data_with_clusters['cluster'] == i]
    temp_df = temp_df.drop(['cluster'],axis=1)
    sum_of_features = temp_df.sum()

    #Create Plot
    sum_of_features.plot.bar(figsize=(16,10),color=colors[i],alpha=0.5)

#Set title and layout
title = _______________
plt.title(title)
plt.xlabel(___________________)
plt.ylabel(___________________)
plt.tight_layout()
__________________

# Empty Markdown for your answers